Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/training-with-deep-learning/how-to-use-estimator/how-to-use-estimator.png)

# How to use Estimator in Azure ML

## Introduction
This tutorial shows how to use the Estimator pattern in Azure Machine Learning SDK. Estimator is a convenient object in Azure Machine Learning that wraps run configuration information to help simplify the tasks of specifying how a script is executed.


## Prerequisite:
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)

Let's get started. First let's import some Python libraries.

In [ ]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## Create an Azure ML experiment
Let's create an experiment named "estimator-test". The script runs will be recorded under this experiment in Azure.

In [ ]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='estimator-test')

## Get default AmlCompute
You can create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, you use default `AmlCompute` as your training compute resource.

In [ ]:
cpu_cluster = ws.get_default_compute_target("CPU")

# use get_status() to get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

Now that you have retrieved the compute target, let's see what the workspace's `compute_targets` property returns.

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

## Use a simple script
We have already created a simple "hello world" script. This is the script that we will submit through the estimator pattern. It prints a hello-world message, and if Azure ML SDK is installed, it will also logs an array of values ([Fibonacci numbers](https://en.wikipedia.org/wiki/Fibonacci_number)). The script takes as input the number of Fibonacci numbers in the sequence to log.

In [ ]:
with open('./dummy_train.py', 'r') as f:
    print(f.read())

## Create A Generic Estimator

First we import the Estimator class and also a widget to visualize a run.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

The simplest estimator is to submit the current folder to the local computer. Estimator by default will attempt to use Docker-based execution. Let's turn that off for now. It then builds a conda environment locally, installs Azure ML SDK in it, and runs your script.

In [ ]:
# use a conda environment, don't use Docker, on local computer
script_params = {
    '--numbers-in-sequence': 10
}
est = Estimator(source_directory='.', script_params=script_params, compute_target='local', entry_script='dummy_train.py', use_docker=False)
run = exp.submit(est)
RunDetails(run).show()

You can also enable Docker and let estimator pick the default CPU image supplied by Azure ML for execution. You can target an AmlCompute cluster (or any other supported compute target types).

In [ ]:
# use a conda environment on default Docker image in an AmlCompute cluster
script_params = {
    '--numbers-in-sequence': 10
}
est = Estimator(source_directory='.', script_params=script_params, compute_target=cpu_cluster, entry_script='dummy_train.py', use_docker=True)
run = exp.submit(est)
RunDetails(run).show()

You can customize the conda environment by adding conda and/or pip packages.

In [ ]:
# add a conda package
script_params = {
    '--numbers-in-sequence': 10
}
est = Estimator(source_directory='.', 
                script_params=script_params, 
                compute_target='local', 
                entry_script='dummy_train.py', 
                use_docker=False, 
                conda_packages=['scikit-learn'])
run = exp.submit(est)
RunDetails(run).show()

You can also specify a custom Docker image for exeution. In this case, you probably want to tell the system not to build a new conda environment for you. Instead, you can specify the path to an existing Python environment in the custom Docker image.

**Note**: since the below example points to the preinstalled Python environment in the miniconda3 image maintained by continuum.io on Docker Hub where Azure ML SDK is not present, the logging metric code is not triggered. But a run history record is still recorded. 

In [ ]:
# use a custom Docker image
from azureml.core.container_registry import ContainerRegistry

# this is an image available in Docker Hub
image_name = 'continuumio/miniconda3'

# you can also point to an image in a private ACR
image_registry_details = ContainerRegistry()
image_registry_details.address = "myregistry.azurecr.io"
image_registry_details.username = "username"
image_registry_details.password = "password"

# don't let the system build a new conda environment
user_managed_dependencies = True

# submit to a local Docker container. if you don't have Docker engine running locally, you can set compute_target to cpu_cluster.
script_params = {
    '--numbers-in-sequence': 10
}
est = Estimator(source_directory='.', 
                script_params=script_params, 
                compute_target='local', 
                entry_script='dummy_train.py',
                custom_docker_image=image_name,
                # uncomment below line to use your private ACR
                #image_registry_details=image_registry_details,
                user_managed=user_managed_dependencies
                )

run = exp.submit(est)
RunDetails(run).show()

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

### Intelligent hyperparameter tuning

The simple "hello world" script above lets the user fix the value of a parameter for the number of Fibonacci numbers in the sequence to log. Similarly, when training models, you can fix values of parameters of the training algorithm itself. E.g. the learning rate, the number of layers, the number of nodes in each layer in a neural network, etc. These adjustable parameters that govern the training process are referred to as the hyperparameters of the model. The goal of hyperparameter tuning is to search across various hyperparameter configurations and find the configuration that results in the best performance.


To demonstrate how Azure Machine Learning can help you automate the process of hyperarameter tuning, we will launch multiple runs with different values for numbers in the sequence. First let's define the parameter space using random sampling.

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice

ps = RandomParameterSampling(
    {
        '--numbers-in-sequence': choice(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20)
    }
)

Next, we will create a new estimator without the above numbers-in-sequence parameter since that will be passed in later. 

In [ ]:
est = Estimator(source_directory='.', script_params={}, compute_target=cpu_cluster, entry_script='dummy_train.py', use_docker=True)

Next, we will look at training metrics and early termination policies. When training a model, users are interested in logging and optimizing certain metrics of the model e.g. maximize the accuracy of the model, or minimize loss. This metric is logged by the training script for each run. In our simple script above, we are logging Fibonacci numbers in a sequence. But a training script could just as easily log other metrics like accuracy or loss, which can be used to evaluate the performance of a given training run.

The intelligent hyperparameter tuning capability in Azure Machine Learning automatically terminates poorly performing runs using an early termination policy. Early termination reduces wastage of compute resources and instead uses these resources for exploring other hyperparameter configurations. In this example, we use the BanditPolicy. This basically states to check the job every 2 iterations. If the primary metric (defined later) falls outside of the top 10% range, Azure ML will terminate the training run. This saves us from continuing to explore hyperparameters that don't show promise of helping reach our target metric.

In [ ]:
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

Now we are ready to configure a run configuration object for hyperparameter tuning. We need to call out the primary metric that we want the experiment to optimize. The name of the primary metric needs to exactly match the name of the metric logged by the training script and we specify that we are looking to maximize this value. Next, we control the resource budget for the experiment by setting the maximum total number of training runs to 10. We also set the maximum number of training runs to run concurrently at 4, which is the same as the number of nodes in our computer cluster.

In [ ]:
hdc = HyperDriveConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='Fibonacci numbers', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=10,
                          max_concurrent_runs=4)

Finally, let's launch the hyperparameter tuning job.

In [ ]:
hdr = exp.submit(config=hdc)
RunDetails(hdr).show()

When all the runs complete, we can find the run with the best performance

In [ ]:
best_run = hdr.get_best_run_by_primary_metric()

We can register the model from the best run and use it to deploy a web service that can be used for Inferencing. Details on how how you can do this can be found in the sample folders for the ohter types of estimators.


## Next Steps
Now you can proceed to explore the other types of estimators, such as TensorFlow estimator, PyTorch estimator, etc. in the sample folder.